In [ ]:
!pip install gradio
!pip install pydub
!pip uninstall spacy weasel
!pip install gradio --upgrade --extra-index-url https://pypi.org/simple/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.9/315.9 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.1 MB/s eta 

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git@main#egg=transformers
!pip install torch


  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-install-90_017ii/transformers_16ccb1583d3948a0aaee4663710f6129
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-install-90_017ii/transformers_16ccb1583d3948a0aaee4663710f6129
  Resolved https://github.com/huggingface/transformers.git to commit 5bf9caa06df47acda751ff5bcef95b937b86c71f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.42.0.dev0-py3-none-any.whl size=9105479 sha256=0ffcc2d860a6ebd9038911526ade3a2cd323505623d3dc3103f7bbe895f901f4
  Stored in directory: /tmp/pip-ephem-wheel-cache-nv6h9ol6/wheels/cf/59/82/6492402e887a68975030bf8c06532260abc16abb7ccd8127cc
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.0
    Uninst

In [ ]:
import gradio as gr
from transformers import pipeline
import numpy as np
from PIL import Image
import time
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

# Define the CLIP pipeline for zero-shot image classification
pipe = pipeline("zero-shot-image-classification", model="openai/clip-vit-base-patch32")

# Function for zero-shot image classification
def zero_shot_classification(image, labels_text, correct_label):
    if not labels_text or not correct_label:
        return "Labels and correct label must be provided.", "", "", "", "", ""

    # Convert image to PIL format
    PIL_image = Image.fromarray(np.uint8(image)).convert('RGB')
    # Split the labels text into a list of labels
    labels = labels_text.split(",")
    if not labels:
        return "At least one label must be provided.", "", "", "", "", ""

    # Perform zero-shot classification
    start_time = time.time()
    res = pipe(images=PIL_image,
               candidate_labels=labels,
               hypothesis_template="This is a photo of a {}")
    end_time = time.time()
    inference_time = end_time - start_time

    # Process the predictions
    predictions = {dic["label"]: dic["score"] for dic in res}
    if not predictions:
        return "No predictions could be made.", "", "", "", "", ""

    max_pred_label = max(predictions, key=predictions.get)
    max_pred_prob = predictions[max_pred_label]

    # Calculate F1 score
    y_true = [1 if label == correct_label else 0 for label in predictions.keys()]
    y_pred = [1 if label == max_pred_label else 0 for label in predictions.keys()]
    f1 = f1_score(y_true, y_pred)

    # Calculate accuracy
    accuracy = 100 if max_pred_label == correct_label else 0

    # Create a bar chart
    fig = plt.figure(figsize=(12, 6))
    plt.bar(predictions.keys(), predictions.values())
    plt.xlabel('Labels')
    plt.ylabel('Confidence Score')
    plt.title('Zero-shot Image Classification Results')
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.tight_layout()
    plt.savefig('chart.png')

    # Format predictions as a string
    predictions_str = '\n'.join(f'{k}: {v:.2f}' for k, v in predictions.items())

    # Return the results
    return predictions_str, f"{inference_time:.2f} seconds", f"F1 Score: {f1:.2f}", f"Accuracy: {accuracy}%", 'chart.png', f"Label with highest predicted probability: {max_pred_label} ({max_pred_prob:.2f})"

# Define the Gradio interface
iface = gr.Interface(fn=zero_shot_classification,
                     inputs=["image", "text", "text"],
                     outputs=[gr.Textbox(label="Predicted Labels"),
                              gr.Textbox(label="Inference Time"),
                              gr.Textbox(label="F1 Score"),
                              gr.Textbox(label="Accuracy"),
                              gr.Image(type="pil", label="Classification Chart"),
                              gr.Textbox(label="Highest Predicted Probability")],
                     description="Add a picture and a list of labels separated by commas",
                     title="Zero-shot Image Classification")

# Launch the Gradio interface
iface.launch(share=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://99ba6cc8402f9ff3f9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
